<a href="https://colab.research.google.com/github/Sathya252/Good_Cab-s_Performance/blob/main/Insights_to_Chief_of_Operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

**Load all datasets**

In [2]:
fact_trips = pd.read_csv('/content/fact_trips.csv')
monthly_target_new_passengers = pd.read_csv('/content/monthly_target_new_passengers.csv')
monthly_target_trips = pd.read_csv('/content/monthly_target_trips.csv')
city_target_passenger_rating = pd.read_csv('/content/city_target_passenger_rating.csv')
dim_city = pd.read_csv('/content/dim_city.csv')
dim_date = pd.read_csv('/content/dim_date.csv')
dim_repeat_trip_distribution = pd.read_csv('/content/dim_repeat_trip_distribution.csv')
fact_passenger_summary = pd.read_csv('/content/fact_passenger_summary.csv')

**Display dataset shapes**

In [3]:
datasets = {
    "fact_trips": fact_trips,
    "monthly_target_new_passengers": monthly_target_new_passengers,
    "monthly_target_trips": monthly_target_trips,
    "city_target_passenger_rating": city_target_passenger_rating,
    "dim_city": dim_city,
    "dim_date": dim_date,
    "dim_repeat_trip_distribution": dim_repeat_trip_distribution,
    "fact_passenger_summary": fact_passenger_summary,
}

for name, df in datasets.items():
    print(f"{name}: {df.shape}")

fact_trips: (425903, 8)
monthly_target_new_passengers: (60, 3)
monthly_target_trips: (60, 3)
city_target_passenger_rating: (10, 2)
dim_city: (10, 2)
dim_date: (182, 4)
dim_repeat_trip_distribution: (540, 4)
fact_passenger_summary: (60, 5)


**Data Exploration**

In [4]:
fact_trips.head()

,trip_id,date,city_id,passenger_type,distance_travelled(km),fare_amount,passenger_rating,driver_rating
0,TRPLUC240113d55de2fb,2024-01-13,UP01,repeated,11,158,5,5
1,TRPVAD240129a3b6dba8,2024-01-29,GJ02,repeated,7,74,5,5
2,TRPCOI240107a42430fb,2024-01-07,TN01,repeated,11,155,8,8
3,TRPKOC240325d7601389,2024-03-25,KL01,repeated,36,427,9,10
4,TRPVIS2406027be97166,2024-06-02,AP01,new,17,265,8,8


In [5]:
for name, df in datasets.items():
    print(f"\n--- {name} ---")
    print(df.info())
    print(df.describe(include='all'))
    print(f"Missing values:\n{df.isnull().sum()}")


--- fact_trips ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425903 entries, 0 to 425902
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   trip_id                 425903 non-null  object
 1   date                    425903 non-null  object
 2   city_id                 425903 non-null  object
 3   passenger_type          425903 non-null  object
 4   distance_travelled(km)  425903 non-null  int64 
 5   fare_amount             425903 non-null  int64 
 6   passenger_rating        425903 non-null  int64 
 7   driver_rating           425903 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 26.0+ MB
None
                     trip_id        date city_id passenger_type  \
count                 425903      425903  425903         425903   
unique                425903         182      10              2   
top     TRPLUC240113d55de2fb  2024-02-03    RJ01       repeated   
freq            

In [6]:
dim_city.city_name.unique()

array(['Jaipur', 'Lucknow', 'Surat', 'Kochi', 'Indore', 'Chandigarh',
       'Vadodara', 'Visakhapatnam', 'Coimbatore', 'Mysore'], dtype=object)

In [7]:
dim_city.city_id.unique()

array(['RJ01', 'UP01', 'GJ01', 'KL01', 'MP01', 'CH01', 'GJ02', 'AP01',
       'TN01', 'KA01'], dtype=object)

In [8]:
fact_trips.passenger_type.value_counts()

,count
passenger_type,
repeated,248905
new,176998


**Data Cleaning**

In [9]:
def clean_data(df):
    # Handle missing values
    if df.isnull().sum().sum() > 0:
        print(f"Missing values in {df.name}:\n{df.isnull().sum()}")
        df.fillna(method='ffill', inplace=True)

    # Remove duplicates
    if df.duplicated().sum() > 0:
        print(f"Duplicates in {df.name}: {df.duplicated().sum()}")
        df.drop_duplicates(inplace=True)

    return df

datasets = {
    "fact_trips": fact_trips,
    "monthly_target_new_passengers": monthly_target_new_passengers,
    "monthly_target_trips": monthly_target_trips,
    "city_target_passenger_rating": city_target_passenger_rating,
    "dim_city": dim_city,
    "dim_date": dim_date,
    "dim_repeat_trip_distribution": dim_repeat_trip_distribution,
    "fact_passenger_summary": fact_passenger_summary,
}

for name, df in datasets.items():
    df.name = name  # Add dataset name for identification
    datasets[name] = clean_data(df)

In [10]:
clean_data(fact_trips)

,trip_id,date,city_id,passenger_type,distance_travelled(km),fare_amount,passenger_rating,driver_rating
0,TRPLUC240113d55de2fb,2024-01-13,UP01,repeated,11,158,5,5
1,TRPVAD240129a3b6dba8,2024-01-29,GJ02,repeated,7,74,5,5
2,TRPCOI240107a42430fb,2024-01-07,TN01,repeated,11,155,8,8
3,TRPKOC240325d7601389,2024-03-25,KL01,repeated,36,427,9,10
4,TRPVIS2406027be97166,2024-06-02,AP01,new,17,265,8,8
...,...,...,...,...,...,...,...,...
425898,TRPLUC2403180b02b4d0,2024-03-18,UP01,repeated,12,134,5,5
425899,TRPVAD2401032679e669,2024-01-03,GJ02,repeated,12,114,7,5
425900,TRPJAI24022578e10280,2024-02-25,RJ01,repeated,26,479,7,10
425901,TRPJAI240401e297ad20,2024-04-01,RJ01,repeated,27,361,7,10


**Data Transfromation**

In [11]:
fact_trips['date'] = pd.to_datetime(fact_trips['date'])

In [12]:
dim_date['start_of_month'] = pd.to_datetime(dim_date['start_of_month'])

In [13]:
dim_city.head()

,city_id,city_name
0,RJ01,Jaipur
1,UP01,Lucknow
2,GJ01,Surat
3,KL01,Kochi
4,MP01,Indore


In [14]:
fact_trips.head()

,trip_id,date,city_id,passenger_type,distance_travelled(km),fare_amount,passenger_rating,driver_rating
0,TRPLUC240113d55de2fb,2024-01-13,UP01,repeated,11,158,5,5
1,TRPVAD240129a3b6dba8,2024-01-29,GJ02,repeated,7,74,5,5
2,TRPCOI240107a42430fb,2024-01-07,TN01,repeated,11,155,8,8
3,TRPKOC240325d7601389,2024-03-25,KL01,repeated,36,427,9,10
4,TRPVIS2406027be97166,2024-06-02,AP01,new,17,265,8,8


In [15]:
fact_trips = fact_trips.merge(dim_city[["city_id", "city_name"]], on="city_id", how="left")
fact_trips.head()

,trip_id,date,city_id,passenger_type,distance_travelled(km),fare_amount,passenger_rating,driver_rating,city_name
0,TRPLUC240113d55de2fb,2024-01-13,UP01,repeated,11,158,5,5,Lucknow
1,TRPVAD240129a3b6dba8,2024-01-29,GJ02,repeated,7,74,5,5,Vadodara
2,TRPCOI240107a42430fb,2024-01-07,TN01,repeated,11,155,8,8,Coimbatore
3,TRPKOC240325d7601389,2024-03-25,KL01,repeated,36,427,9,10,Kochi
4,TRPVIS2406027be97166,2024-06-02,AP01,new,17,265,8,8,Visakhapatnam


In [16]:
fact_trips.head()

,trip_id,date,city_id,passenger_type,distance_travelled(km),fare_amount,passenger_rating,driver_rating,city_name
0,TRPLUC240113d55de2fb,2024-01-13,UP01,repeated,11,158,5,5,Lucknow
1,TRPVAD240129a3b6dba8,2024-01-29,GJ02,repeated,7,74,5,5,Vadodara
2,TRPCOI240107a42430fb,2024-01-07,TN01,repeated,11,155,8,8,Coimbatore
3,TRPKOC240325d7601389,2024-03-25,KL01,repeated,36,427,9,10,Kochi
4,TRPVIS2406027be97166,2024-06-02,AP01,new,17,265,8,8,Visakhapatnam


In [17]:
city_revenue = fact_trips.groupby('city_name')['fare_amount'].sum().reset_index()
city_revenue.rename(columns={'fare_amount': 'total_revenue'}, inplace=True)
city_revenue

,city_name,total_revenue
0,Chandigarh,11058401
1,Coimbatore,3523992
2,Indore,7635228
3,Jaipur,37207497
4,Kochi,16997596
5,Lucknow,9463551
6,Mysore,4054745
7,Surat,6431599
8,Vadodara,3797200
9,Visakhapatnam,8018282


# **Business Request 1: City-Level Fare and Trip Summary Report**

**Data Cleaning**

In [18]:
fact_trips = fact_trips.dropna(subset=['city_id', 'fare_amount', 'distance_travelled(km)'])
fact_trips = fact_trips[fact_trips['distance_travelled(km)'] > 0]

**Data Transformation**

In [19]:
city_summary = fact_trips.groupby('city_name').agg(
    total_trips=('trip_id', 'count'),
    avg_fare_per_km=('fare_amount', lambda x: (x.sum() / fact_trips['distance_travelled(km)'].sum())),
    avg_fare_per_trip=('fare_amount', 'mean')
).reset_index()
print(city_summary)

       city_name  total_trips  avg_fare_per_km  avg_fare_per_trip
0     Chandigarh        38981         1.357472         283.686950
1     Coimbatore        21104         0.432587         166.982183
2         Indore        42456         0.937261         179.838609
3         Jaipur        76888         4.567399         483.918128
4          Kochi        50702         2.086537         335.245079
5        Lucknow        64299         1.161696         147.180376
6         Mysore        16238         0.497739         249.707168
7          Surat        54843         0.789510         117.272925
8       Vadodara        32026         0.466125         118.566165
9  Visakhapatnam        28366         0.984283         282.672284


**Insight Generation**

In [20]:
total_trips = city_summary['total_trips'].sum()
city_summary['% contribution_to_total_trips'] = (city_summary['total_trips'] / total_trips) * 100
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
print(city_summary)

       city_name  total_trips  avg_fare_per_km  avg_fare_per_trip  % contribution_to_total_trips
0     Chandigarh        38981         1.357472         283.686950                       9.152554
1     Coimbatore        21104         0.432587         166.982183                       4.955119
2         Indore        42456         0.937261         179.838609                       9.968467
3         Jaipur        76888         4.567399         483.918128                      18.052937
4          Kochi        50702         2.086537         335.245079                      11.904589
5        Lucknow        64299         1.161696         147.180376                      15.097100
6         Mysore        16238         0.497739         249.707168                       3.812605
7          Surat        54843         0.789510         117.272925                      12.876876
8       Vadodara        32026         0.466125         118.566165                       7.519553
9  Visakhapatnam        28366 

# **Business Request 2: Monthly City-Level Trips Target Performance Report**

**Data Cleaning**

In [21]:
fact_trips['date'] = pd.to_datetime(fact_trips['date'])
fact_trips['month'] = fact_trips['date'].dt.to_period('M')
monthly_target_trips['month'] = pd.to_datetime(monthly_target_trips['month']).dt.to_period('M')

**Data Transformation**

In [22]:
monthly_trips = fact_trips.groupby(['city_id', 'month']).size().reset_index(name='actual_trips')
monthly_trips = monthly_trips.merge(monthly_target_trips, on=['city_id', 'month'], how='left')

monthly_trips['performance_status'] = monthly_trips.apply(
    lambda x: "Above Target" if x['actual_trips'] > x['total_target_trips'] else "Below Target", axis=1
)
monthly_trips['% difference'] = ((monthly_trips['actual_trips'] - monthly_trips['total_target_trips']) /
                                 monthly_trips['total_target_trips']) * 100

**Insights Generation**

In [23]:
monthly_trips.head()

,city_id,month,actual_trips,total_target_trips,performance_status,% difference
0,AP01,2024-01,4468,4500,Below Target,-0.711111
1,AP01,2024-02,4793,4500,Above Target,6.511111
2,AP01,2024-03,4877,4500,Above Target,8.377778
3,AP01,2024-04,4938,5000,Below Target,-1.240000
4,AP01,2024-05,4812,5000,Below Target,-3.760000


# **Business Request 3: City-Level Repeat Passenger Trip Frequency Report**

**Data Cleaning**

In [24]:
dim_repeat_trip_distribution['repeat_passenger_count'] = pd.to_numeric(dim_repeat_trip_distribution['repeat_passenger_count'], errors='coerce')
dim_repeat_trip_distribution.dropna(inplace=True)

**Data Transformation**

In [25]:
repeat_trips = dim_repeat_trip_distribution.pivot_table(
    index='city_id',
    columns='trip_count',
    values='repeat_passenger_count',
    aggfunc='sum',
    fill_value=0
)

**Insight Generation**

In [26]:
repeat_trips_percentage = repeat_trips.div(repeat_trips.sum(axis=1), axis=0) * 100
repeat_trips_percentage.reset_index(inplace=True)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
print(repeat_trips_percentage)

trip_count city_id  10-Trips    2-Trips    3-Trips    4-Trips    5-Trips    6-Trips    7-Trips   8-Trips   9-Trips
0             AP01  0.920125  51.252937  24.960846   9.984338   5.442443   3.191073   1.977291  1.389977  0.880971
1             CH01  1.794872  32.307692  19.250493  15.739645  12.209073   7.416174   5.483235  3.471400  2.327416
2             GJ01  1.354480   9.759204  14.262561  16.554758  19.749942  18.453346  11.889326  6.239870  1.736513
3             GJ02  1.610676   9.871146  14.173953  16.520939  18.062586  19.075012  12.862402  5.775426  2.047860
4             KA01  0.473934  48.747461  24.441435  12.728504   5.822613   4.062288   1.760325  1.421801  0.541638
5             KL01  0.813008  47.665880  24.350905  11.814844   6.477839   3.907684   2.111199  1.652242  1.206399
6             MP01  1.510532  34.340355  22.685698  13.400776  10.338137   6.845898   5.238359  3.256652  2.383592
7             RJ01  0.970874  50.144598  20.729188  12.115265   6.290023   4.131

# **Business Request 4: Identify Cities with Highest and Lowest Total New Passengers**

**Data Cleaning**

In [27]:
fact_passenger_summary = fact_passenger_summary[fact_passenger_summary['new_passengers'] >= 0]
fact_passenger_summary.head()

,month,city_id,new_passengers,repeat_passengers,total_passengers
0,2024-01-01,AP01,2513,650,3163
1,2024-01-01,CH01,3920,720,4640
2,2024-01-01,GJ01,2432,1184,3616
3,2024-01-01,GJ02,2089,544,2633
4,2024-01-01,KA01,1957,172,2129


**Data Transformation**

In [28]:
city_new_passengers = fact_passenger_summary.groupby('city_id')['new_passengers'].sum().reset_index()
city_new_passengers = city_new_passengers.merge(dim_city, on='city_id', how='left')

city_new_passengers['rank'] = city_new_passengers['new_passengers'].rank(ascending=False)
city_new_passengers['city_category'] = city_new_passengers['rank'].apply(
    lambda x: "Top 3" if x <= 3 else "Bottom 3" if x > len(city_new_passengers) - 3 else "Other"
)

**Insights Genration**

In [29]:
city_new_passengers.sort_values(by='new_passengers', ascending=False)

,city_id,new_passengers,city_name,rank,city_category
7,RJ01,45856,Jaipur,1.0,Top 3
5,KL01,26416,Kochi,2.0,Top 3
1,CH01,18908,Chandigarh,3.0,Top 3
9,UP01,16260,Lucknow,4.0,Other
6,MP01,14863,Indore,5.0,Other
0,AP01,12747,Visakhapatnam,6.0,Other
4,KA01,11681,Mysore,7.0,Other
2,GJ01,11626,Surat,8.0,Bottom 3
3,GJ02,10127,Vadodara,9.0,Bottom 3
8,TN01,8514,Coimbatore,10.0,Bottom 3


# **Business Request 5: Identify Month with Highest Revenue for Each City**

**Data Cleaning**

In [30]:
fact_trips = fact_trips[fact_trips['fare_amount'] >= 0]

**Data Transformation**

In [31]:
fact_trips['month'] = fact_trips['date'].dt.to_period('M')
monthly_revenue = fact_trips.groupby(['city_id', 'month'])['fare_amount'].sum().reset_index()

highest_revenue = monthly_revenue.loc[monthly_revenue.groupby('city_id')['fare_amount'].idxmax()]
highest_revenue['% contribution'] = highest_revenue.groupby('city_id')['fare_amount'].transform(
    lambda x: (x / x.sum()) * 100
)

**Insight Generation**

In [32]:
highest_revenue.sort_index(ascending=False)

,city_id,month,fare_amount,% contribution
55,UP01,2024-02,1777269,100.0
51,TN01,2024-04,612431,100.0
43,RJ01,2024-02,7747202,100.0
40,MP01,2024-05,1380996,100.0
34,KL01,2024-05,3333746,100.0
28,KA01,2024-05,745170,100.0
21,GJ02,2024-04,706250,100.0
15,GJ01,2024-04,1154909,100.0
7,CH01,2024-02,2108290,100.0
3,AP01,2024-04,1390682,100.0


# **Business Request 6: Repeat Passenger Rate Analysis**

**Data Cleaning**

In [33]:
fact_passenger_summary = fact_passenger_summary[(fact_passenger_summary['repeat_passengers'] >= 0) &(fact_passenger_summary['total_passengers'] >= 0)]

In [34]:
fact_passenger_summary.head()

,month,city_id,new_passengers,repeat_passengers,total_passengers
0,2024-01-01,AP01,2513,650,3163
1,2024-01-01,CH01,3920,720,4640
2,2024-01-01,GJ01,2432,1184,3616
3,2024-01-01,GJ02,2089,544,2633
4,2024-01-01,KA01,1957,172,2129


**Data Transformation**

In [35]:
fact_passenger_summary['monthly_repeat_passenger_rate'] = (
    fact_passenger_summary['repeat_passengers'] / fact_passenger_summary['total_passengers'] * 100
)
city_repeat_rate = fact_passenger_summary.groupby('city_id').agg(
    total_passengers=('total_passengers', 'sum'),
    repeat_passengers=('repeat_passengers', 'sum')
)
city_repeat_rate['city_repeat_passenger_rate'] = round(
    city_repeat_rate['repeat_passengers'] / city_repeat_rate['total_passengers'] * 100.2
)

**Insights Generation**

In [36]:
fact_passenger_summary.head()

,month,city_id,new_passengers,repeat_passengers,total_passengers,monthly_repeat_passenger_rate
0,2024-01-01,AP01,2513,650,3163,20.550111
1,2024-01-01,CH01,3920,720,4640,15.517241
2,2024-01-01,GJ01,2432,1184,3616,32.743363
3,2024-01-01,GJ02,2089,544,2633,20.660843
4,2024-01-01,KA01,1957,172,2129,8.078910


In [37]:
city_repeat_rate.head()

,total_passengers,repeat_passengers,city_repeat_passenger_rate
city_id,,,
AP01,17855,5108,29.0
CH01,23978,5070,21.0
GJ01,20264,8638,43.0
GJ02,14473,4346,30.0
KA01,13158,1477,11.0
